# Text to SQL

In [3]:
from langchain_community.tools import QuerySQLDatabaseTool
from langchain_community.utilities import SQLDatabase
from langchain.chains.sql_database.query import create_sql_query_chain
from langchain_ollama import ChatOllama

Let's download a sample database to test the Text to SQL model. Datasource obtained using following [documentation](https://python.langchain.com/docs/tutorials/sql_qa/).

In [1]:
# save result in data folder
! curl -s "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql" | sqlite3 ../data/Chinook.db

In [4]:
db = SQLDatabase.from_uri("sqlite:///../data/Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [5]:
llm = ChatOllama(model="llama3.1", temperature=0)

# convert question to sql query
# method doc https://python.langchain.com/api_reference/langchain/chains/langchain.chains.sql_database.query.create_sql_query_chain.html
write_query = create_sql_query_chain(llm, db)

# Execute SQL query
execute_query = QuerySQLDatabaseTool(db=db)

# combined
chain = write_query | execute_query

chain.invoke({"question": "How many employees are there"})


'[(8,)]'

We first convert the user's query to a SQL query. Then we execute that query on our database. 

> **Warning**:
> Executing arbitrary SQL queries on your database generated by an LLM from user input is dangerous in a production application. To use these ideas in production, you need to consider a number of security measures to reduce the risk of unintended queries being run in the database. Here are some ideas:
> - Run the queries on our database with a user with read-only permissions.
> - The database user running the queries should have access only to the tables you wish to make availabe for querying.
> - Add a time-out to the queries run by this application; this would ensure that even if an expensive query is generated, it is canceled before taking up too many of your database resources.

This are some ideas, as LLM vulnerabilities are not well understood yet. Let's go back to the [main file](../README.md/#Adding-memory-to-chatbots-using-LangGraph).